In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np



In [120]:
# Carregar os dados
df = pd.read_csv('C:/Users/julia/OneDrive/Documentos/Ciencia_de_dados/Modelo_Previsao_Compras/Dados_Ficticios_Vendas.csv', low_memory=False)



In [121]:
# Converter a coluna 'DateSale' para datetime, especificando o formato 'YYYY-MM-DD'
df['DateSale'] = pd.to_datetime(df['DateSale'], format='%Y-%m-%d', errors='coerce')

# Verificar se a conversão foi bem-sucedida e lidar com dados faltantes
if df['DateSale'].isnull().any():
    print("Existem datas inválidas ou ausentes que foram convertidas para NaT.")
    print(df[df['DateSale'].isnull()])
    df = df.dropna(subset=['DateSale'])



In [122]:
# Ordenar por 'product_id' e 'DateSale'
df = df.sort_values(by=['product_id', 'DateSale'])



In [123]:
# Calcular o intervalo entre compras
df['Intervalo_Dias'] = df.groupby('product_id')['DateSale'].diff().dt.days



In [124]:
# Preencher NaNs com a média dos intervalos
df['Intervalo_Dias'].fillna(df['Intervalo_Dias'].mean(), inplace=True)



In [125]:
# Feature engineering: Usar 'price_x' e 'quantity' como características
X = df[['price_x', 'quantity']]
y_intervalo = df['Intervalo_Dias']
y_quantity = df['quantity']  # Usando a quantidade atual para prever a quantidade



In [126]:
# Dividir dados em treino e teste
X_train, X_test, y_train_intervalo, y_test_intervalo = train_test_split(X, y_intervalo, test_size=0.2, random_state=42)
_, _, y_train_quantity, y_test_quantity = train_test_split(X, y_quantity, test_size=0.2, random_state=42)



In [127]:
# Treinar o modelo para Intervalo_Dias
model_intervalo = RandomForestRegressor(n_estimators=100, random_state=42)
model_intervalo.fit(X_train, y_train_intervalo)



RandomForestRegressor(random_state=42)

In [128]:
# Treinar o modelo para quantidade
model_quantity = RandomForestRegressor(n_estimators=100, random_state=42)
model_quantity.fit(X_train, y_train_quantity)



RandomForestRegressor(random_state=42)

In [129]:
# Prever o intervalo de dias e a quantidade para o conjunto completo
df['Previsao_Intervalo_Dias'] = model_intervalo.predict(X)
df['Previsao_Quantidade'] = model_quantity.predict(X)



In [130]:
# Calcular a data prevista para a próxima compra
df['Data_Previsao_Compra'] = df['DateSale'] + pd.to_timedelta(df['Previsao_Intervalo_Dias'], unit='d')



In [131]:
# Obter a última data de compra para cada produto
ultima_compra = df.groupby('product_id')['DateSale'].max().reset_index()
ultima_compra.rename(columns={'DateSale': 'Ultima_Data_Compra'}, inplace=True)



In [132]:
# Obter a média dos intervalos de compra e quantidade para cada produto
media_intervalos = df.groupby('product_id')['Previsao_Intervalo_Dias'].mean().reset_index()
media_intervalos.rename(columns={'Previsao_Intervalo_Dias': 'Media_Intervalo_Dias'}, inplace=True)
media_quantidade = df.groupby('product_id')['Previsao_Quantidade'].mean().reset_index()
media_quantidade.rename(columns={'Previsao_Quantidade': 'Media_Quantidade'}, inplace=True)



In [133]:
# Prever a próxima data de compra usando a média do intervalo
media_intervalos['Data_Previsao_Compra'] = pd.to_datetime(ultima_compra['Ultima_Data_Compra']) + pd.to_timedelta(media_intervalos['Media_Intervalo_Dias'], unit='d')



In [134]:
# Selecionar e unir as previsões com as últimas datas de compra
resultado = pd.merge(ultima_compra[['product_id', 'Ultima_Data_Compra']], 
                     media_intervalos[['product_id', 'Data_Previsao_Compra']],
                     on='product_id')
resultado = pd.merge(resultado, media_quantidade[['product_id', 'Media_Quantidade']], on='product_id')



In [135]:
# Exibir o resultado
print(resultado[['product_id', 'Data_Previsao_Compra', 'Media_Quantidade']])


     product_id          Data_Previsao_Compra  Media_Quantidade
0           201 2024-08-21 05:00:10.492151146        187.676702
1           202 2024-08-26 03:19:11.442914492        223.678996
2           203 2024-08-25 02:19:05.415872615        326.227124
3           204 2024-08-26 02:57:43.424784541        183.358027
4           205 2024-08-25 02:46:45.972289223        230.232996
5           206 2024-08-26 00:37:18.621151746        207.354596
6           207 2024-08-22 04:21:35.747767869        130.219735
7           208 2024-08-21 03:40:30.068458633        274.204624
8           209 2024-08-23 04:00:57.925901868        203.594188
9           210 2024-08-24 03:15:55.156202409        310.915372
10          211 2024-08-23 03:40:33.979135324        234.404356
11          212 2024-08-25 02:33:49.218835291        317.385561
12          213 2024-08-23 02:35:28.975535335        313.721111
13          214 2024-08-26 04:36:59.980929048        208.930831
14          215 2024-08-22 02:41:39.1012